# Overview

Terrain is the web API that the CyVese Discovery Environment uses to perform all of its tasks. Terrain is a REST-like web API. That is, it acts like a traditional web server in many ways. Requests that are sent to terrain are formatted just like a request that your browser might make in order to open a web page. Responses from Terrain are typically very similar to responses that your browser might receive from a request. The primary difference is the response format. The format that your browser receives from a web server will typically be formatted as a a web page whereas a response from Terrain will usually be a document formatted in JavaScript Object Notation (JSON). The primary reason for this difference is that JSON is easier for computers to read than HTML, and it's easier for humans to read than another common format in use on the internet, XML. If you're unfamiliar with JSON, DigitalOcean has this great [introduction](https://www.digitalocean.com/community/tutorials/an-introduction-to-json) that will quickly bring you up to speed.

Without further ado, let's give Terrain a try! 

## Prerequisites

Before we can use Terrain, we'll need some prerequisites. The `requests` library is an easy way to make requests to web services in Python, so let's import it so that we can use it later. It'll also be necessary to prompt for a password and to pretty-print some data structures.

In [ ]:
import getpass
import pprint
import requests

## The Welcome Message

Terrain has a bit of a goofy welcome message that you can see by hitting its root endpoint. The original purpose of the message was for developers to have an easy way to verify that Terrain was in fact running and responsive. We still use this endpoint on a regular basis, but now the calls to it are automated. For some reason, the computers don't find it as amusing as the humans do.

In [ ]:
r = requests.get("https://de.cyverse.org/terrain/")
r.raise_for_status()
print(r.text)

## Authentication

To do anything interesting in terrain, you have to authenticate to the service. Terrain uses Apereo's [Central Authentication Service](https://apereo.github.io/cas/5.3.x/) OAuth provider for authentication. Typically you'd need to regsiter a client to allow you to authenticate directly to CAS, which can be a lot of work. For this reason, Terrain has an endpoint that you can use to obtain a token indirectly using Terrain's own OAuth client. But in order to get the access token, we'll need your password.

In [ ]:
print("Username: ", end='', flush=True)
username = input()
print("Password: ", end='', flush=True)
password = getpass.getpass()

And now we can obtain an access token.

In [ ]:
r = requests.get("https://de.cyverse.org/terrain/token", auth=(username, password))
r.raise_for_status()
token = r.json()['access_token']
auth_headers = {"Authorization": "Bearer " + token}

## Searching for Apps

_Now_ we can do something interesting. Suppose you've got an eight-hundred-word essay to write, and you want to use the DE to make sure that your essay is long enough. You can use one of the DE's word count apps to find out!

In [ ]:
query_params = {"search": "DE Word Count"}
r = requests.get("https://de.cyverse.org/terrain/apps", headers=auth_headers, params=query_params)
r.raise_for_status()
pprint.pprint(r.json())

## Getting Analysis Submission Information

Before you can submit an analysis to the Discovery environment, you'll need to know how to pass parameters to the app. The endpoint to get information on how to submit an analysis for a specific app is `GET /terrain/apps/{system-id}/{app-id}`, where the System ID and app ID are fields obtained from the analysis listing returned by the previous endpoint. Since we still have the response body from the app lookup, we can extract the information we need from it.

In [ ]:
app_listing = r.json()["apps"][0]
system_id = app_listing["system_id"]
app_id = app_listing["id"]
print("System ID: ", system_id)
print("App ID: ", app_id)

Now calling the analysis submission information endpoint gives us the information we need.

In [ ]:
url = "https://de.cyverse.org/terrain/apps/{0}/{1}".format(system_id, app_id)
r = requests.get(url, headers=auth_headers)
r.raise_for_status()
pprint.pprint(r.json())

The output from this endpoint deserves a little explanation. At the top level, we have the basic app information such as the name, ID, and description of the app. The top level also contains a list labeled `groups`. These groups provide a way to place related parameters on the same panel in the app launch window in the DE. Each group contains a list of parameters, and the parameters themselves provide the information we need to submit the job.

The primary piece of information that we're going to need from this file is the parameter ID for the input file name. We may as well grab it now.

In [ ]:
parameter_id = r.json()["groups"][0]["parameters"][0]["id"]
print("Parameter ID: ", parameter_id)

## Building the Analysis Submission Request Body

The analysis submission endpoint is the first endpoint we've encountered so far that has a request body, and this request body needs to be formatted correctly for the analysis submission to succeed. The request body looks something like this:

``` json
{
  "config": {},
  "name": "string",
  "app_id": "string",
  "system_id": "string",
  "debug": false,
  "output_dir": "string",
  "notify": true
}
```

I've taken the liberty of removing optional fields from the example. The fields are defined as follows:

| Parameter Name | Description                                                                       |
| -------------- | --------------------------------------------------------------------------------- |
| config         | A map from parameter ID to parameter value.                                       |
| name           | The name of the analysis.                                                         |
| app_id         | The app ID from the submission information above.                                 |
| system_id      | The system ID from the submission information above.                              |
| debug          | This parameter can be used to enable debugging, which isn't necessary.            |
| output_dir     | The path to the folder in the data store where the output files should be placed. |
| notify         | This parameter can be used to enable or disable job status update notifications.  |

So now we have to plug in the values. Suppose the essay whose words we have to count is at this path in the CyVerse data store: `/iplant/home/shared/workshop_material/terrain_intro/essay.txt`. And now we have enough information to format the request body.

Keep in mind that the request body below is written in Python rather than JSON, so it will look slightly different from the JSON request body listed above. The `requests` library will convert this Python object to a JSON object for us before sending the request to terrain.

In [ ]:
request_body = {
    "config": {
        parameter_id: "/iplant/home/shared/workshop_material/terrain_intro/essay.txt"
    },
    "name": "essay-word-count",
    "app_id": app_id,
    "system_id": system_id,
    "debug": False,
    "output_dir": "/iplant/home/" + username + "/analyses",
    "notify": True
}
pprint.pprint(request_body)

Now we can finally submit the request.

In [ ]:
r = requests.post("https://de.cyverse.org/terrain/analyses", headers=auth_headers, json=request_body)
pprint.pprint(r.json())